# <center>ĐỒ ÁN CUỐI KỲ: KHOA HỌC DỮ LIỆU ỨNG DỤNG</center>

**Đề tài:** Dự đoán kết quả học tập của học sinh dựa trên việc chơi game

**Nhóm:** Bumblebee - Nhóm 2

**Thành viên**

| MSSV     | Họ và tên       | Github                      |
|----------|-----------------|-----------------------------|
| 19120511 | Võ Văn Hiếu     | https://github.com/vv-hieu  |
| 19120526 | Huỳnh Đức Huy   | https://github.com/KaiKush  |
| 19120539 | Vương Thế Khang | https://github.com/vt-khang |


**Github repository:** https://github.com/vt-khang/KHDLUD-FinalProject

## Import

### Import libraries

In [4]:
!pip install dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Import libraries
import dask.dataframe as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Import data

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"crawll","key":"797100d166c0a72ebefebec89c8318f6"}'}

In [4]:
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!kaggle competitions download -c predict-student-performance-from-game-play

 99% 959M/968M [00:04<00:00, 244MB/s]
100% 968M/968M [00:04<00:00, 237MB/s]


In [6]:
!mkdir data
!unzip -q predict-student-performance-from-game-play.zip -d /content/data
!rm predict-student-performance-from-game-play.zip

mkdir: cannot create directory ‘data’: File exists
replace /content/data/jo_wilder/__init__.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/data/jo_wilder/competition.cpython-37m-x86_64-linux-gnu.so? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## A. THU THẬP DỮ LIỆU

### Mục tiêu đề tài

Dự đoán hiệu suất học tập của học sinh trong các trò chơi giáo dục dựa trên thời gian thực. Từ đó phát triển mô hình học máy dựa trên một tập dữ liệu lớn các nhật ký trò chơi được thu thập từ các học sinh. Trong đồ án này sử dụng trò chơi giáo dục trực tuyến [Jo Wilder](https://pbswisconsineducation.org/jowilder/play-the-game) để theo dõi quá trình học tập của học sinh.

### Giới thiệu bộ dữ liệu

Bộ dữ liệu của cuộc thi là dữ liệu về chuỗi thời gian được ghi lại từ trò chơi giáo dục trực tuyến [Jo Wilder](https://pbswisconsineducation.org/jowilder/play-the-game). Trò chơi được chia thành 4 chương, sau mỗi chương người chơi sẽ được trả lời một số câu hỏi. Có tổng cộng 18 câu hỏi trong trò chơi. Mục tiêu của cuộc thi là sử dụng bộ dữ liệu từ những câu hỏi trên để xác định xem người chơi có trả lời đúng câu hỏi hay không, từ đó có thể đưa ra các kết luận về hiệu suất học tập của học sinh.

Các file dữ liệu này bao gồm:
- **train.csv** - tập train
- **test.csv** - tập test
- **sample_submission.csv** - định dạng file dùng để nộp trên Kaggle
- **train_labels.csv** - câu trả lời cho 18 câu hỏi của mỗi session trong tập train

Cuộc thi: [Predict Student Performance from Game Play](https://www.kaggle.com/competitions/predict-student-performance-from-game-play)

Nhà tổ chức cuộc thi: [Field Day Lab](https://fielddaylab.wisc.edu)

Nhà tài trợ: [The Learning Agency Lab](https://www.the-learning-agency-lab.com)

License: OPEN SOURCE - MIT

### Ứng dụng và ý nghĩa

- Thúc đẩy nghiên cứu về các phương pháp giáo dục, dạy học, tiếp thu kiến thức, ... từ các trò chơi giáo dục.
- Hỗ trợ các nhà phát triển phần mềm trò chơi (Game Developer) để tạo ra thêm nhiều trải nghiệm học tập có hiệu quả hơn cho học sinh.

## B. KHÁM PHÁ DỮ LIỆU

Đầu tiên, ta sẽ xem thử bộ dữ liệu trông như thế nào. Đọc dữ liệu từ file "survey.csv" và lưu vào dataframe `df`.

In [7]:
df = pd.read_csv('/content/data/train.csv')
df.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [12]:
df.dtypes

session_id          int64
index               int64
elapsed_time        int64
event_name         object
name               object
level               int64
page              float64
room_coor_x       float64
room_coor_y       float64
screen_coor_x     float64
screen_coor_y     float64
hover_duration    float64
text               object
fqid               object
room_fqid          object
text_fqid          object
fullscreen          int64
hq                  int64
music               int64
level_group        object
dtype: object

In [18]:
df['screen_coor_x'].max().compute()

KeyboardInterrupt: ignored